In [1]:
import datetime
import os.path
from collections import OrderedDict

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
SCOPES = ['https://www.googleapis.com/auth/calendar']

In [3]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [4]:
def get_calendar_list(calendar_service):
    calendar_list = calendar_service.calendarList().list().execute()
    ret = calendar_list.get('items', [])
    return [ {'id':x['id'],'summary':x['summary']} for x in ret ]

def get_calendar_events(calendar_service, calendar_id, max_results=10):
    now = datetime.datetime.utcnow().isoformat() + 'Z'
    events_result = calendar_service.events().list(
        calendarId=calendar_id,
        timeMin=now,
        maxResults=max_results,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    return events_result.get('items', [])

def get_new_title(title):
    global MAP
    for a,b in MAP.items():
        title = title.replace(a,b)
    for a,b in MAP.items():
        title = title.replace(" "+b,b)
    return title

def update_event_title(calendar_service, event_id, new_title):
    event = calendar_service.events().get(calendarId='primary', eventId=event_id).execute()
    event['summary'] = new_title
    updated_event = calendar_service.events().update(calendarId='primary', eventId=event_id, body=event).execute()

In [5]:
MAP = OrderedDict()

In [6]:
with open('map.txt') as f:
    lines = f.readlines()
    lines = [x.split() for x in lines]
    for a,b in lines:
        MAP[a] = b

In [7]:
service = build('calendar', 'v3', credentials=creds)

In [8]:
calendar_id = "chaiosjang@gmail.com"

In [9]:
events = get_calendar_events(service, calendar_id, max_results=100)
# print(events)
events = [x for x in events if '#' in x['summary']]
# print(events)

In [10]:
for i in range(0,len(events)):
    e = events[i]
    dt = e['start']
    if   'date'     in dt: dt = dt['date']
    elif 'dateTime' in dt: dt = dt['dateTime']
    old_title = e['summary']
    new_title = get_new_title(old_title)
    e['summary'] = new_title
    dt = "{:<{}}".format(dt, 25)
    print("%s %s" % (dt, old_title))
    print("%s %s" % (">"*26, new_title))
    updated_event = service.events().update(calendarId=calendar_id, eventId=e['id'], body=e).execute()

2023-08-20                 #회사 #해외 APAN56
>>>>>>>>>>>>>>>>>>>>>>>>>> 🏢✈️ APAN56
2023-08-26T10:00:00+09:00  #육아 텃밭 가꾸기
>>>>>>>>>>>>>>>>>>>>>>>>>> 👶 텃밭 가꾸기
2023-08-28T10:30:00+09:00  #회사 #티타임 Mazahir
>>>>>>>>>>>>>>>>>>>>>>>>>> 🏢☕ Mazahir
2023-09-12T08:30:00+09:00  #건강검진 을지대 (위내시경 제외)
>>>>>>>>>>>>>>>>>>>>>>>>>> 🏥 을지대 (위내시경 제외)
2023-10-11                 #회사 #출장 EOST Open Source Tech Day
>>>>>>>>>>>>>>>>>>>>>>>>>> 🏢🚌 EOST Open Source Tech Day
2023-11-04                 #건강검진 을지대 (위내시경)
>>>>>>>>>>>>>>>>>>>>>>>>>> 🏥 을지대 (위내시경)
